In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
import csv

# Load data
ordini_per_cliente = json.load(open('data/utils/ordini_per_client.json'))       
tot_per_cliente = json.load(open('data/utils/tot_per_client.json')) 
quantity = pd.read_csv('data/utils/quantity_per_client.csv')
 
print(f'prima {quantity.shape}')

job_machines = json.load(open('data/utils/articoli_macchine.json'))
job_list_temp = [job for job in job_machines] 

#Filtraggio degli articoli che non hanno dati presenti su "lista_articoli"
lista_articoli = {}
with open('data/valid/lista_articoli.csv', newline='') as csvfile:
    csv_data = csv.reader(csvfile, delimiter=',', quotechar='"')
    #row[0] è il codarticolo
    for idx, row in enumerate(csv_data):
        if job_list_temp.count(row[0]) > 0:
            lista_articoli[row[0]] = {}
            lista_articoli[row[0]]['kg_ora'] = float(row[6])
            lista_articoli[row[0]]['ciclo'] = int(float(row[9]))
            lista_articoli[row[0]]['ore_levata'] = int(float(row[10]))
            lista_articoli[row[0]]['kg_levata'] = int(float(row[6]) * float(row[10]))



#Filtraggio degli ordini che NON presentano almeno una associazione macchina
o_p_c_filtrata = {}
for cliente in ordini_per_cliente:
    dict_ordini = ordini_per_cliente[cliente] 
    for ordine in dict_ordini:
        if lista_articoli.get(ordine) is not None:
            if cliente not in o_p_c_filtrata:
                o_p_c_filtrata[cliente] = {}
            # Append the new value to the list associated with the key
            o_p_c_filtrata[cliente][ordine] = ordini_per_cliente[cliente][ordine]
        else:
            print(f'per cliente: {cliente}, ordine: {ordine}, non trovato in lista_articoli')

#Filtraggio delle statistiche per cliente solo per i clienti che presentano ordini
t_p_c_filtrata = {}
for cliente in tot_per_cliente:
    if o_p_c_filtrata.get(cliente) is not None:
        t_p_c_filtrata[cliente] = tot_per_cliente[cliente]

#Filtraggio quantità per estrarre le statistiche e correzione di queste per avere solo le statistiche degli ordini che hanno tutti i dati(associazioni macchina,...)
quantity_filtrata = {'Cliente': [], 'cod articolo': [], 'mean': [], 'std': [], 'mean_differenza_giorni': []}
for index, row in quantity.iterrows():
    # print(f"Row {index}: {row.to_dict()}")
    if t_p_c_filtrata.get(row['Cliente']) is not None and lista_articoli.get(row['cod articolo']) is not None:
        # Values corrections
        kg_levata = lista_articoli[row['cod articolo']]['kg_levata']
        mean = row['mean']
        mean_differenza_giorni = row['mean_differenza_giorni']
        if mean / kg_levata < 0.9:
            giorni_levata = lista_articoli[row['cod articolo']]['ore_levata'] / 24
            if giorni_levata > mean_differenza_giorni:
                mean_differenza_giorni = giorni_levata 
            mean = int(kg_levata * 0.9)         

        # Variable assignement
        quantity_filtrata['Cliente'].append(row['Cliente'])
        quantity_filtrata['cod articolo'].append(row['cod articolo'])
        quantity_filtrata['mean'].append(mean)
        quantity_filtrata['std'].append(row['std'])
        quantity_filtrata['mean_differenza_giorni'].append(mean_differenza_giorni)

quantity = pd.DataFrame(quantity_filtrata)
print(f'dopo {quantity.shape}')

print(f'prima job_list_temp: {len(job_list_temp)}, dopo job_list: {len(lista_articoli)}')
print(f'prima: {len(ordini_per_cliente)}, dopo {len(o_p_c_filtrata)}')
print(ordini_per_cliente['GT 2000'])
print(o_p_c_filtrata.get('GT 2000'))

print(f'prima: {len(tot_per_cliente)}, dopo {len(t_p_c_filtrata)}')

# Utils
clienti = list(o_p_c_filtrata.keys())               # Lista di clienti
tot = sum(t_p_c_filtrata.values())                     # Totale ordini per cliente nello storico
norm = {k: v/tot for k,v in t_p_c_filtrata.items()}    # Probabilità per cliente di comparire nello storico

prima (316, 5)
per cliente: ALPIN CORAP, ordine: 23505DN1, non trovato in lista_articoli
per cliente: ALPIN CORAP, ordine: 23505DN2, non trovato in lista_articoli
per cliente: AMBROFIBRE, ordine: X21-4480S+ZN, non trovato in lista_articoli
per cliente: AMBROFIBRE, ordine: X23-2282S+ZN, non trovato in lista_articoli
per cliente: AMBROFIBRE, ordine: X24-2201S+ZN, non trovato in lista_articoli
per cliente: APPLE, ordine: S18-60042V4DN, non trovato in lista_articoli
per cliente: APPLE, ordine: S18-60042V2DN, non trovato in lista_articoli
per cliente: APPLE, ordine: S15-60008V3DN2, non trovato in lista_articoli
per cliente: APPLE, ordine: S600A16ZN100, non trovato in lista_articoli
per cliente: ATHENA, ordine: 4457ZN2, non trovato in lista_articoli
per cliente: ATHENA, ordine: X22-7810S-ZN354, non trovato in lista_articoli
per cliente: AUSSCO, ordine: S15-60008V3DN2, non trovato in lista_articoli
per cliente: AUSSCO, ordine: S15-60008V3DNcolorat, non trovato in lista_articoli
per cliente: B

In [5]:
import pandas as pd

def get_dates_between(start_date_str, end_date_str, days_to_subtract):
    # Convert input strings to pandas datetime objects
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)
    
    # Calculate the "start_offset" by subtracting days from the "end_date"
    start_offset = end_date - pd.Timedelta(days=days_to_subtract)
    
    # Generate the list of dates between start_date and start_offset (inclusive)
    date_range = pd.date_range(start=start_date, end=start_offset).strftime("%Y-%m-%d").tolist()
    
    return date_range


In [ ]:
### Generate synthetic data

## params
num_samples = 5     # Number of samples to generate

# Set to True to generate data starting from today (orders in entrata oggi)
today = False       

# Altrimenti imposta un range
start = '2024-11-20'
end = '2025-03-15'

new_data = []

# Loop to generate synthetic data with progress bar
for _ in tqdm(range(num_samples), desc="Generating data"):

    # Scegli cliente in modo uniforme
    i = np.round(np.random.uniform(0, len(clienti) - 1))
    cliente = clienti[int(i)]
    prob = norm[cliente]
    num = int(np.round(prob * num_samples))                     # Numero di ordini da generare per il cliente
    rows = quantity[quantity['Cliente'] == cliente].shape[0]
    if rows > 0:                                                # Se il cliente ha articoli nel dataset
        articoli = quantity[quantity['Cliente'] == cliente]     # Filtra articoli per cliente
        
        for _ in range(num + 1):
            art_index = int(np.round(np.random.uniform(0, rows - 1)))   # Scegli un articolo in modo uniforme
            art = articoli.iloc[art_index]

            articolo = art['cod articolo']              # Nome dell'articolo
            quantity_mean = art['mean']                 # Media della quantità ordinata                
            quantity_std = art['std']                   # Deviazione standard della quantità ordinata
            giorni = art['mean_differenza_giorni']      # media(consegna - ordine) | articolo, cliente

            # Ensure kg is always positive
            kg = np.round(np.random.normal(quantity_mean, quantity_std))
            if kg <= 0:                                     # Se la quantità è negativa, 
                kg = quantity_mean                              # prendi la media
            
            if today:
                data_inizio = pd.Timestamp.today().date()
            # else:
            #     date_range = pd.date_range(start=start, end=end)
            #     data_inizio = np.random.choice(date_range)
            #     data_inizio = pd.Timestamp(data_inizio).date()
            else:
                offset_inizio = get_dates_between(start, end, giorni)
                data_inizio = np.random.choice(pd.DatetimeIndex(offset_inizio))
                data_inizio = pd.Timestamp(data_inizio).date()

            # Calculate the end date
            data_fine = data_inizio + pd.Timedelta(days=giorni + np.random.randint(0,giorni))
            data_fine = pd.Timestamp(data_fine).date()

            # Append the generated data to new_data list
            new_data.append({                               # Inserisci come dizionario in una lista di ordini
                'cliente': cliente,
                'cod_articolo': articolo,
                'quantity': kg,
                'data inserimento': data_inizio,
                'data consegna': data_fine
            })

columns = ['cliente', 'cod_articolo', 'quantity', 'data inserimento', 'data consegna']
res = pd.DataFrame(data=new_data, columns=columns)

res.to_csv('data/new_orders.csv', index=False)
res

Generating data:   0%|          | 0/5 [00:00<?, ?it/s]

Generating data: 100%|██████████| 5/5 [00:00<00:00, 129.89it/s]


,cliente,cod_articolo,quantity,data inserimento,data consegna
0,DIAMANT,39502DN-14446,170.0,2025-02-07,2025-04-04
1,CALZE LUISA,2246ZN 10800,198.0,2025-01-30,2025-03-20
2,SANYLEG,33002DN,385.0,2025-01-28,2025-04-19
3,SANYLEG,62001DN2,170.0,2025-01-25,2025-03-11
4,SCORTA,4407DN,472.0,2024-11-21,2024-12-28
5,GREENFLEX BV,s22-700011DN,149.0,2024-12-03,2025-01-03
